# 5.2

In [12]:
import sympy
from sympy import Matrix, Symbol, diag, ones, zeros, integrate
from sympy.abc import xi, m, x, l, omega, E, A
sympy.init_printing()

水平放置的等截面直杆绕支点O在水平面内以角速度$\omega$旋转，
为求出中点C和支点O处的应力，用两个3结点直杆单元进行有限元分析，
分析后分别用总体磨平、单元磨平、分片磨平方法进行应力处理，
得到C点和O点的应力值，
并和解析解相比较。

## 坐标系

局部坐标采用$\xi=-1\sim 1$。

## 位移插值函数

In [4]:
interpolation_function = [
    ((xi - 0) * (xi - 1)) / ((-1 - 0) * (-1 - 1)),
    ((xi + 1) * (xi - 1)) / ((0 + 1) * (0 - 1)),
    ((xi + 1) * (xi - 0)) / ((1 + 1) * (1 - 0)),
]
interpolation_function

In [5]:
N = [f for f in interpolation_function]
N = Matrix(N).T
N

⎡ξ⋅(ξ - 1)                    ξ⋅(ξ + 1)⎤
⎢─────────  -(ξ - 1)⋅(ξ + 1)  ─────────⎥
⎣    2                            2    ⎦

## 应变矩阵

In [6]:
B = [f.diff(xi) for f in interpolation_function]
B = Matrix(B).T
B

[ξ - 1/2  -2⋅ξ  ξ + 1/2]

## 弹性矩阵

In [7]:
D = E * A
D

## 应力矩阵

应力矩阵

In [8]:
S = D * B
S

[A⋅E⋅(ξ - 1/2)  -2⋅A⋅E⋅ξ  A⋅E⋅(ξ + 1/2)]

## 单元刚度矩阵

In [100]:
Ke = integrate(2*E*A/l*N.T.diff(xi)*N.diff(xi), (xi,-1,1))
Ke

⎡ 7⋅A⋅E   -8⋅A⋅E     A⋅E  ⎤
⎢ ─────   ───────    ───  ⎥
⎢  3⋅l      3⋅l      3⋅l  ⎥
⎢                         ⎥
⎢-8⋅A⋅E   16⋅A⋅E   -8⋅A⋅E ⎥
⎢───────  ──────   ───────⎥
⎢  3⋅l     3⋅l       3⋅l  ⎥
⎢                         ⎥
⎢  A⋅E    -8⋅A⋅E    7⋅A⋅E ⎥
⎢  ───    ───────   ───── ⎥
⎣  3⋅l      3⋅l      3⋅l  ⎦

## 结点编号

从左至右分别为1、2、3、4、5。

## 总刚矩阵

In [101]:
K = zeros(5, 5)
K[0:3, 0:3] += Ke
K[2:5, 2:5] += Ke
K

⎡ 7⋅A⋅E   -8⋅A⋅E     A⋅E                    ⎤
⎢ ─────   ───────    ───       0        0   ⎥
⎢  3⋅l      3⋅l      3⋅l                    ⎥
⎢                                           ⎥
⎢-8⋅A⋅E   16⋅A⋅E   -8⋅A⋅E                   ⎥
⎢───────  ──────   ───────     0        0   ⎥
⎢  3⋅l     3⋅l       3⋅l                    ⎥
⎢                                           ⎥
⎢  A⋅E    -8⋅A⋅E   14⋅A⋅E   -8⋅A⋅E     A⋅E  ⎥
⎢  ───    ───────  ──────   ───────    ───  ⎥
⎢  3⋅l      3⋅l     3⋅l       3⋅l      3⋅l  ⎥
⎢                                           ⎥
⎢                  -8⋅A⋅E   16⋅A⋅E   -8⋅A⋅E ⎥
⎢   0        0     ───────  ──────   ───────⎥
⎢                    3⋅l     3⋅l       3⋅l  ⎥
⎢                                           ⎥
⎢                    A⋅E    -8⋅A⋅E    7⋅A⋅E ⎥
⎢   0        0       ───    ───────   ───── ⎥
⎣                    3⋅l      3⋅l      3⋅l  ⎦

## 荷载

In [18]:
p = - m * x * omega ** 2
p

### 单元1荷载

In [19]:
p1 = p.subs({x: (xi+1)/4*l})
p1

In [20]:
jacobian = ((xi + 1) / 4 * l).diff(xi)
jacobian

In [52]:
P1 = integrate(N * p1 * jacobian, (xi, -1, 1))
P1 = Matrix(P1)
P1

⎡     2    2     2    2 ⎤
⎢   -l ⋅m⋅ω    -l ⋅m⋅ω  ⎥
⎢0  ─────────  ─────────⎥
⎣       12         24   ⎦

### 单元2荷载

In [27]:
p2 = p.subs({x: (xi+3)/4*l})
p2

In [28]:
jacobian = ((xi + 3) / 4 * l).diff(xi)
jacobian

In [51]:
P2 = integrate(N * p2 * jacobian, (xi, -1, 1))
P2 = Matrix(P2)
P2

⎡  2    2     2    2     2    2 ⎤
⎢-l ⋅m⋅ω    -l ⋅m⋅ω    -l ⋅m⋅ω  ⎥
⎢─────────  ─────────  ─────────⎥
⎣    24         4          12   ⎦

### 总荷载

In [77]:
P = zeros(1, 5)
P[0, 0:3] += P1
P[0, 2:5] += P2
P

⎡     2    2     2    2     2    2     2    2 ⎤
⎢   -l ⋅m⋅ω    -l ⋅m⋅ω    -l ⋅m⋅ω    -l ⋅m⋅ω  ⎥
⎢0  ─────────  ─────────  ─────────  ─────────⎥
⎣       12         12         4          12   ⎦

## 位移

In [91]:
Ptmp = P.copy()
Ptmp.col_del(0)
Ptmp = Ptmp.T
Ptmp

⎡  2    2 ⎤
⎢-l ⋅m⋅ω  ⎥
⎢─────────⎥
⎢    12   ⎥
⎢         ⎥
⎢  2    2 ⎥
⎢-l ⋅m⋅ω  ⎥
⎢─────────⎥
⎢    12   ⎥
⎢         ⎥
⎢  2    2 ⎥
⎢-l ⋅m⋅ω  ⎥
⎢─────────⎥
⎢    4    ⎥
⎢         ⎥
⎢  2    2 ⎥
⎢-l ⋅m⋅ω  ⎥
⎢─────────⎥
⎣    12   ⎦

In [94]:
Ktmp = K.copy()
Ktmp.col_del(0)
Ktmp.row_del(0)
Ktmp

⎡16⋅A⋅E   -8⋅A⋅E                   ⎤
⎢──────   ───────     0        0   ⎥
⎢ 3⋅l       3⋅l                    ⎥
⎢                                  ⎥
⎢-8⋅A⋅E   14⋅A⋅E   -8⋅A⋅E     A⋅E  ⎥
⎢───────  ──────   ───────    ───  ⎥
⎢  3⋅l     3⋅l       3⋅l      3⋅l  ⎥
⎢                                  ⎥
⎢         -8⋅A⋅E   16⋅A⋅E   -8⋅A⋅E ⎥
⎢   0     ───────  ──────   ───────⎥
⎢           3⋅l     3⋅l       3⋅l  ⎥
⎢                                  ⎥
⎢           A⋅E    -8⋅A⋅E    7⋅A⋅E ⎥
⎢   0       ───    ───────   ───── ⎥
⎣           3⋅l      3⋅l      3⋅l  ⎦

In [113]:
u = zeros(5, 1)
u[1:, 0] = Ktmp.inv() * Ptmp
u

⎡     0      ⎤
⎢            ⎥
⎢     3    2 ⎥
⎢-47⋅l ⋅m⋅ω  ⎥
⎢────────────⎥
⎢  192⋅A⋅E   ⎥
⎢            ⎥
⎢     3    2 ⎥
⎢-11⋅l ⋅m⋅ω  ⎥
⎢────────────⎥
⎢   24⋅A⋅E   ⎥
⎢            ⎥
⎢     3    2 ⎥
⎢-39⋅l ⋅m⋅ω  ⎥
⎢────────────⎥
⎢   64⋅A⋅E   ⎥
⎢            ⎥
⎢    3    2  ⎥
⎢-2⋅l ⋅m⋅ω   ⎥
⎢─────────── ⎥
⎣   3⋅A⋅E    ⎦

## 应力

### 单元1应力

In [119]:
sigma1 = S * u[0:3,0]
sigma1 = sigma1[0]
sigma1

### 单元2应力

In [122]:
sigma2 = S * u[2:5,0]
sigma2 = sigma2[0]
sigma2

### 各点的应力

In [126]:
sigma = Matrix([
    sigma1.subs({xi: -1}),
    sigma1.subs({xi: 0}),
    sigma1.subs({xi: 1}),
    sigma2.subs({xi: 0}),
    sigma2.subs({xi: 1}),
])
sigma

⎡     3    2 ⎤
⎢-25⋅l ⋅m⋅ω  ⎥
⎢────────────⎥
⎢     96     ⎥
⎢            ⎥
⎢     3    2 ⎥
⎢-11⋅l ⋅m⋅ω  ⎥
⎢────────────⎥
⎢     48     ⎥
⎢            ⎥
⎢     3    2 ⎥
⎢-19⋅l ⋅m⋅ω  ⎥
⎢────────────⎥
⎢     96     ⎥
⎢            ⎥
⎢    3    2  ⎥
⎢-5⋅l ⋅m⋅ω   ⎥
⎢─────────── ⎥
⎢     48     ⎥
⎢            ⎥
⎢   3    2   ⎥
⎢ -l ⋅m⋅ω    ⎥
⎢ ─────────  ⎥
⎣     96     ⎦